In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
from scipy.stats import skew, kurtosis

For initial development purposes, I'm using the mnist dataset of handdrawn digits.
This should be removed for actual project

In [ ]:
mnist = tf.keras.datasets.mnist #collect data

#split data into training and testing data
(x_train, y_train), (x_test, y_test) = mnist.load_data()